# What's in an Avocado Toast: A Supply Chain Analysis

![](avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

Apply your data manipulation and analysis skills to investigate the supply chain of ingredients for making avocado toast in the U.K. Your task is to determine the following information:

- The name of the most common country of origin for each of the three key ingredients: avocados, olive oil, and sourdough.
Store the most common country of origin for each ingredient in the following variables: `top_avocado_origin`, `top_olive_oil_origin`, and `top_sourdough_origin`. 
Ensure that the country names contain only letters (A-Z) and spaces, with no hyphens or other characters.

To focus your analysis, subset each of the DataFrames to include only these relevant columns: 'code', 'lc', 'productnameen', 'quantity', 'servingsize', 'packagingtags', 'brands', 'brandstags', 'categoriestags', 'labelstags', 'countries', 'countriestags', 'origins', 'origins_tags'.

After completing this project, feel free to explore other questions using the food data!

In [11]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.width", 1000)

avocado = pd.read_csv("data/avocado.csv", sep="\t")
print(avocado.head())

            code  lc product_name_de product_name_el                    product_name_en product_name_es product_name_fi        product_name_fr product_name_id product_name_it  ... off:ecoscore_data.adjustments.packaging.non_recyclable_and_non_biodegradable_materials off:ecoscore_data.adjustments.production_system.value off:ecoscore_data.adjustments.threatened_species.value sources_fields:org-database-usda:available_date sources_fields:org-database-usda:fdc_category sources_fields:org-database-usda:fdc_data_source sources_fields:org-database-usda:fdc_id sources_fields:org-database-usda:modified_date sources_fields:org-database-usda:publication_date                                     data_sources
0  0059749979702  fr             NaN             NaN                                NaN             NaN             NaN  Naturalia Avocado Oil             NaN             NaN  ...                                                1.0                                                                 

In [5]:
subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
avocado = avocado[subset_columns]

# Gather relevant categories data for avocados
with open("data/relevant_avocado_categories.txt", "r") as file:
    relevant_avocado_categories = file.read().splitlines()
    file.close()

avocado['categories_list'] = avocado['categories_tags'].str.split(',')
avocado = avocado.dropna(subset = 'categories_list')

# Filter a DataFrame based on a column of lists
avocado = avocado[avocado['categories_list'].apply(lambda x: any([i for i in x if i in relevant_avocado_categories]))]

print(avocado)

               code  lc      product_name_en   quantity serving_size packaging_tags                        brands                   brands_tags                                    categories_tags                                        labels_tags       countries     countries_tags            origins       origins_tags                                    categories_list
5     3662994002063  fr                  NaN   3 fruits          NaN            NaN  la compagnie des fruits mûrs  la-compagnie-des-fruits-murs  en:plant-based-foods-and-beverages,en:plant-ba...                                                NaN          France          en:france                NaN                NaN  [en:plant-based-foods-and-beverages, en:plant-...
6     8437013031011  fr                  NaN       1 kg          NaN            NaN                           NaN                           NaN  en:plant-based-foods-and-beverages,en:plant-ba...                                                NaN          Franc

In [ ]:
### Where do most avocados come from?
# Filter DataFrame for UK data
avocados_uk = avocado[(avocado['countries']=='United Kingdom')]

# Find most common country for avocado origin
avocado_origin = (avocados_uk['origins_tags'].value_counts().index[0])
avocado_origin = avocado_origin.lstrip("en:")

avocado_origin

'peru'

In [7]:
def read_and_filter_data(filename, relevant_categories):
    df = pd.read_csv('data/' + filename, sep='\t')

    subset_columns = [ 'code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
    df = df[subset_columns]

    # Split tags into lists
    df['categories_list'] = df['categories_tags'].str.split(',')

    # Drop rows with null categories data
    df = df.dropna(subset = 'categories_list')

    # Filter data for relevant categories
    df = df[df['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]
        
    # Filter data for the UK
    df_uk = df[(df['countries']=='United Kingdom')]

    # Find top origin country string with the highest count
    top_origin_string = (df_uk['origins_tags'].value_counts().index[0])

    # Clean up top origin country string
    top_origin_country = top_origin_string.lstrip("en:")
    top_origin_country = top_origin_country.replace('-', ' ')

    print(f'**{filename[:-4]} origins**','\n', top_origin_country, '\n')

    print ("Top origin country: ", top_origin_country)
    print ("\n")

    # End of function - return top origin country for this ingredient
    return top_origin_country

In [8]:
top_avocado_origin = read_and_filter_data('avocado.csv', relevant_avocado_categories)

print(top_avocado_origin)

**avocado origins** 
 peru 

Top origin country:  peru


peru


In [12]:
with open("data/relevant_olive_oil_categories.txt", "r") as file:
    relevant_olive_oil_categories = file.read().splitlines()
    file.close()

top_olive_oil_origin = read_and_filter_data('olive_oil.csv',relevant_olive_oil_categories)
print(top_olive_oil_origin)

**olive_oil origins** 
 greece 

Top origin country:  greece


greece


In [10]:
with open("data/relevant_sourdough_categories.txt", "r") as file:
    relevant_sourdough_categories = file.read().splitlines()
    file.close()

# Call user-defined function on sourdough.csv
top_sourdough_origin = read_and_filter_data('sourdough.csv',relevant_sourdough_categories)
print(top_sourdough_origin)

**sourdough origins** 
 united kingdom 

Top origin country:  united kingdom


united kingdom
